![rmotr](https://www.wingoo-solutions.fr/194/105-logo-asi.gif)
<hr style="margin-bottom: 20px;">

# Rapport sur l'accidentalité en france
#### Source: Ministère de l'intérieur sur data.gouv.fr
<hr style="margin-bottom: 20px;">

## Data preparation: Importer et nettoyer les données

### Lecture et jointu des fichiers

In [2]:
# Fichiers caracteristiques
all_files_caracteristiques = glob.glob(r'C:\DataSources\caracteristiques' + "/*.csv") # Lister les fichiers contenus dans le path
li_caracteristiques = [] # Initialiser la lister de lécture
for filename in all_files_caracteristiques: # Boucler sur les fichiers en les lisant
    df = pd.read_csv(filename, index_col=None, header=0, encoding = "latin") # Bien spécifier encoding = "latin"
    li_caracteristiques.append(df) 
frame_caracteristiques = pd.concat(li_caracteristiques, axis=0, ignore_index=True) # Créer un dataframe à partir de la liste des fichiers lus
print('====== {} lignes importées à partir des fichiers de caracteristiques'.format(frame_caracteristiques['Num_Acc'].count()))


# Lecture lieux 
all_files_lieux = glob.glob(r'C:\DataSources\lieux' + "/*.csv")
li_lieux = []
for filename in all_files_lieux:
    df = pd.read_csv(filename, index_col=None, header=0, encoding = "latin", sep=',', low_memory=False)
    li_lieux.append(df)
frame_lieux = pd.concat(li_lieux, axis=0, ignore_index=True)
print('====== {} lignes importées à partir des fichiers de lieux'.format(frame_lieux['Num_Acc'].count()))

# Lecture usagers 
all_files_usagers = glob.glob(r'C:\DataSources\usagers' + "/*.csv")
li_usagers = []
for filename in all_files_usagers:
    df = pd.read_csv(filename, index_col=None, header=0, encoding = "latin", sep=',', low_memory=False)
    li_usagers.append(df)
frame_usagers = pd.concat(li_usagers, axis=0, ignore_index=True)
print('====== {} lignes importées à partir des fichiers de usagers'.format(frame_usagers['Num_Acc'].count()))

#### Jointure des accidents
frame_accident = pd.merge(frame_caracteristiques, frame_lieux, on = 'Num_Acc')
df_acc_usr = pd.merge(frame_accident, frame_usagers, on = 'Num_Acc')
df_acc_usr['grav'] = df_acc_usr['grav'].map(lambda x: 1 if x == 2 else 0 )  # Accident morte = 1 autre accident = 0
print(f'====== Jointure des dataframe caracteristiques, lieux et usagers \n')

====== 958469 lignes importées à partir des fichiers de caracteristiques
====== 958469 lignes importées à partir des fichiers de lieux
====== 2142195 lignes importées à partir des fichiers de usagers
====== Jointure des dataframe caracteristiques, lieux et usagers 



### Nettoyage des données

#### Mise en forme des dates

In [3]:
# Mise en forme de la date
## ¨pit avoir toutes les années tout format de quatre charactères. Ex: ( 5 -> 005 -> 05 -> 2005)  , (16 -> 0016, 16 -> 2016)
df_acc_usr['an']  = '20' + ('00' + df_acc_usr['an'].map(str) ).str[-2:]

## Pour avoir tout les mois sous format de deux charactères. Ex: ( 1 -> 001 -> 01) , (13 -> 0013 -> 13)
df_acc_usr['mois']  = ('00' + df_acc_usr['mois'].map(str)).str[-2:]

## Pour avoir tout les jours sous format de deux charactères. Ex: ( 3 -> 003 -> 03) , (15 -> 0015 -> 15)
df_acc_usr['jour']  = ('00' + df_acc_usr['jour'].map(str) ).str[-2:]

## Pour avoir tout les jours sous format de quatre charactères. Ex: ( 3 -> 003 -> 003) , (1250 -> 001250 -> 1250) , (230 -> 00230 -> 0230)
df_acc_usr['hrmn']  = ('00' + df_acc_usr['hrmn'].map(str) ).str[-4:]

## Ajout de la colonne Date
df_acc_usr['Date'] = pd.to_datetime(df_acc_usr['an'] + df_acc_usr['mois'] + df_acc_usr['jour'] + df_acc_usr['hrmn'] , format='%Y%m%d%H%M')
print(df_acc_usr['Date'].head(5))

0   2017-01-11 18:20:00
1   2017-01-11 18:20:00
2   2017-01-11 18:20:00
3   2017-02-13 16:30:00
4   2017-02-13 16:30:00
Name: Date, dtype: datetime64[ns]


#### Calcule des ages

In [4]:
df_acc_usr['age'] = ( np.int64(df_acc_usr['an']) - df_acc_usr['an_nais'] )
print(df_acc_usr['age'].head(5))

0    49.0
1    44.0
2    50.0
3    64.0
4    57.0
Name: age, dtype: float64


#### Supression des colonnes non utilisées

In [5]:
# Supression des colonnes inutiles
df_acc_usr = df_acc_usr.drop(['an', 'mois', 'jour', 'hrmn', 'com', 'adr'
                 , 'gps', 'lat', 'long', 'dep', 'an_nais', 'num_veh'
                              , 'env1', 'place', 'trajet'], axis=1)



#### Changement des types de données

In [6]:
df_acc_usr = df_acc_usr.astype({
    'lum': 'category', 'agg': 'category', 'int': 'category', 
    'atm': 'category', 'col': 'category', 'catr': 'category',
    'circ': 'category', 'vosp': 'category', 'prof': 'category',
    'plan': 'category', 'surf': 'category', 'infra': 'category', 
    'situ': 'category', 'catu': 'category', 'grav': 'category',
    'sexe': 'category'
})

### Analyse des données

#### Aperçu des données 

In [7]:
df_acc_usr.describe(percentiles=None, include='category')

,lum,agg,int,atm,col,catr,circ,vosp,prof,plan,surf,infra,situ,catu,grav,sexe
count,2142195,2142195,2142195,2142046.0,2142152.0,2142193.0,2138626.0,2135842.0,2137752.0,2136789.0,2137788.0,2129595.0,2130588.0,2142195,2142195,2142195
unique,5,2,10,9.0,7.0,8.0,5.0,4.0,5.0,5.0,10.0,8.0,6.0,4,2,2
top,1,2,1,1.0,3.0,4.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1,0,1
freq,1476913,1409485,1508764,1731553.0,646254.0,1020816.0,1348599.0,2004172.0,1630969.0,1653317.0,1673260.0,1888746.0,1837742.0,1594260,2084614,1439318


In [8]:
df_acc_usr.describe(percentiles=None, exclude=['category', 'datetime', 'object'])

,Num_Acc,v1,nbv,pr,pr1,lartpc,larrout,secu,locp,actp,etatp,age
count,2.142195e+06,750230.000000,2.135950e+06,1.121537e+06,1.117644e+06,2.014575e+06,2.018811e+06,2.085658e+06,2.085834e+06,2.085733e+06,2.085777e+06,2.139777e+06
mean,2.010923e+11,0.079216,2.107546e+00,9.236276e+01,2.899833e+02,5.575486e+00,5.985202e+01,1.796288e+01,2.307082e-01,2.834946e-01,1.053665e-01,3.748199e+01
std,4.081452e+08,0.608558,1.528709e+00,4.428746e+02,3.586551e+02,2.332773e+01,6.585246e+01,1.929059e+01,8.830024e-01,1.063983e+00,3.849168e-01,1.826791e+01
min,2.005000e+11,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.007001e+11,0.000000,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.300000e+01
50%,2.011000e+11,0.000000,2.000000e+00,7.000000e+00,1.450000e+02,0.000000e+00,6.000000e+01,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,3.400000e+01
75%,2.014001e+11,0.000000,2.000000e+00,3.000000e+01,5.000000e+02,0.000000e+00,8.000000e+01,2.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01
max,2.018001e+11,9.000000,9.900000e+01,9.929000e+03,9.540000e+03,9.600000e+02,9.990000e+02,9.300000e+01,8.000000e+00,9.000000e+00,3.000000e+00,1.090000e+02


#### Taux de valeurs manquantes

In [9]:
df_missing_values = pd.DataFrame(df_acc_usr.isnull().sum())[0]
df_missing_values = pd.DataFrame({'colonne':df_missing_values.index, 'nbr_manquant':df_missing_values.values})
nbr_lignes = df_acc_usr['Num_Acc'].count()
df_missing_values["%_missing"] = ((df_missing_values['nbr_manquant'] / nbr_lignes )* 100 ).round(decimals=2)

#### Supression des colonnes avec trop de valeurs manquantes

In [10]:
df_acc_usr = df_acc_usr.drop(['v1', 'v2', 'pr', 'pr1', 'voie', 'larrout', 'lartpc'
               , 'nbv', 'locp', 'secu', 'actp', 'etatp'] , axis=1)

#### Imputation des données

In [12]:
# Remplacer les valeurs manquantes de l'age par la moyenne
df_acc_usr['age'].fillna(df_acc_usr['age'].mean(), inplace=True)

# Je n'ai pas pu remplacer les valeurs manquantes des attribtuts quali de la même façon à cause d'un bug panda https://github.com/pandas-dev/pandas/issues/35731
# J'ai utilisé l'imputer de sklearner à la place
imp = SimpleImputer(strategy='most_frequent')
imputed_df_acc_usr = imp.fit_transform(df_acc_usr)

# De l'imputation résulte un numpy_darray, il faudra le transformer en dataframe pour continuer les analyses
imputed_df_acc_usr = pd.DataFrame(imputed_df_acc_usr, columns=['lum', 'agg', 'int', 'atm', 'col', 
                                                               'catr', 'circ', 'vosp','prof', 'plan', 'surf', 
                                                               'infra', 'situ', 'catu','grav', 'sexe', 
                                                               'Date', 'age'])

# Verifier que toutes les valeurs manquantes ont été remplacées
df_missing_values = pd.DataFrame(imputed_df_acc_usr.isnull().sum())[0]
df_missing_values = pd.DataFrame({'colonne':df_missing_values.index, 'nbr_manquant':df_missing_values.values})
nbr_lignes = df_acc_usr['Num_Acc'].count()
df_missing_values["%_missing"] = ((df_missing_values['nbr_manquant'] / nbr_lignes )* 100 ).round(decimals=2)


AssertionError: Number of manager items must equal union of block items
# manager items: 18, # tot_items: 19

In [ ]:
imputed_df_acc_usr.head(5)

### Création des dataframe utilisés pour l'analyse

#### Accidents par année

In [ ]:
AccidentDates = df_acc_usr['Date']
AccidentByYear = AccidentDates.groupby([df_acc_usr['Date'].dt.year.rename('annee')]).agg({'count'})
AccidentByYear.head(5)

####  Accidents par mois

In [ ]:
AccidentByMonth = pd.DataFrame({'Date' :  df_acc_usr['Date']  , 'Accidents' : 1})
AccidentByMonth['year_month'] = pd.to_datetime(((AccidentByMonth['Date'].dt.year).map(str) + (AccidentByMonth['Date'].dt.month).map(str)) , format='%Y%m')
AccidentByMonth = AccidentByMonth.groupby('year_month' , as_index=False)['Accidents'].sum()

AccidentByMonth['annee'] = AccidentByMonth['year_month'].dt.year
AccidentByMonth['mois'] = AccidentByMonth['year_month'].dt.month_name()
del AccidentByMonth['year_month']
AccidentByMonth_pivoted = AccidentByMonth.pivot(index='mois', columns='annee', values='Accidents')
AccidentByMonth

### Machin learning

#### prédiction de la gravité de l'accident

In [ ]:
# Choix des colonnes à utiliser pour la prédiction
feature_cols = list(df_acc_usr.columns) # Lister toutes les colonnes
feature_cols.remove('Num_Acc') # Colonne non utilisée
feature_cols.remove('Date') # Colonne non utilisée
feature_cols.remove('grav') # Colonne non utilisée

# DF des observations
X = imputed_df_acc_usr[feature_cols] 

# DF cible
y = df_acc_usr['grav'] 

# Découper le dataset en aprentissage et teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# Création d'un objet arbre de décision, critère de division est l'entropy
clf = DecisionTreeClassifier(max_depth=6, criterion = "entropy")
# Aprentissage d'arbre de décision
clf = clf.fit(X_train,y_train)

# Prédiction sur le dataset
y_pred = clf.predict(X_test)

print('En utilisant les colonnes  \n{}'.format(feature_cols))
print("nous avont pu prédit le type d accident avec une accuracy de :",metrics.accuracy_score(y_test, y_pred))